In [10]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [11]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-04-30 18:14:35--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.28MB/s    in 0.2s    

2021-04-30 18:14:36 (5.28 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [12]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [13]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Shoes_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
shoes_data_df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Shoes_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
shoes_data_df.show(15)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18069663|R3P2HIOQCIN5ZU|B000XB31C0|     265024781|Minnetonka Men's ...|           Shoes|          1|            0|          0|   N|                Y|                   .|Do not buy: reall...| 2015-08-31|
|         US|   16251825|R12VVR0WH5Q24V|B00CFYZH5W|     259035853|Teva Men's Pajaro...|           Shoes|          5|    

In [14]:
# Drop duplicates and null values 
print(shoes_data_df.count())
shoes_data_df = shoes_data_df.dropna()
print(shoes_data_df.count())
shoes_data_df = shoes_data_df.dropDuplicates()
print(shoes_data_df.count())

4366916
4366324
4366324


In [15]:
# View the Schema
shoes_data_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [9]:
# Create new data frames for tables in Schema
review_id_table = shoes_data_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.show(15)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1000E0AHJ5E1V|   12731835|B00E9PFHPK|     257352271| 2013-10-04|
|R100A5CHCA9GMG|   12123247|B00DZI79R6|     575104294| 2015-04-10|
|R100A968SZR4GH|    2481979|B00AMU1Z4I|     706513982| 2014-08-04|
|R100HVIG9VEABL|   10761726|B003GIS6FE|     634629189| 2011-05-14|
|R100ZMBT0DGFFH|   32268353|B0061RUKCE|     195365109| 2014-08-20|
|R101BSLZ8K12T6|    1796899|B0023NTWN8|     300200501| 2015-01-23|
|R101I5DOJAUSZ7|   23422005|B00QAKVP4E|      77952021| 2015-04-01|
|R101VVQ9LXBADV|   35169932|B00B9TXEUY|     898577149| 2014-05-30|
|R101VY19JHVABI|   37230772|B005CFVSZY|     268889520| 2013-10-06|
|R1026R3QQEM02Q|   28698079|B000TH4T42|     361609926| 2015-08-20|
|R102IW3CHZL3F4|   41657996|B001G8FD2K|     805441169| 2012-12-05|
|R102JNG6P6AHY7|   49541857|B0090S9KJK|     807355156| 2013-02

In [16]:
products = shoes_data_df.select(["product_id", "product_title"])
products.show(15)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00E9PFHPK|Qupid Ponder-06 N...|
|B00DZI79R6|REFRESH CHERRY-02...|
|B00AMU1Z4I|Nike Women's Luna...|
|B003GIS6FE|Keen Concordia Co...|
|B0061RUKCE|LensCovers Sungla...|
|B0023NTWN8|Kamik Rocket Cold...|
|B00QAKVP4E|Gabor Lanzarote 7...|
|B00B9TXEUY|Merrell Women's J...|
|B005CFVSZY|Crocs Women's Adr...|
|B000TH4T42|Bogs Men's Classi...|
|B001G8FD2K|KEEN Kootenay Win...|
|B0090S9KJK|Qupid PROUD-09 Cu...|
|B004MOFUBY|Stride Rite Flora...|
|B0026ZQJF2|Bearpaw Women's K...|
|B00CYBLN3G|Skechers Sport Wo...|
+----------+--------------------+
only showing top 15 rows



In [17]:
customers = shoes_data_df.select(["customer_id"])
customers.show(15)

+-----------+
|customer_id|
+-----------+
|   12731835|
|   12123247|
|    2481979|
|   10761726|
|   32268353|
|    1796899|
|   23422005|
|   35169932|
|   37230772|
|   28698079|
|   41657996|
|   49541857|
|   14784542|
|   15510683|
|   36227396|
+-----------+
only showing top 15 rows



In [18]:
customers = customers.groupby(['customer_id']).count()
customers.show(15)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   32605012|    2|
|    7458575|    1|
|   33739415|   22|
|   52884959|    3|
|   18141648|    1|
|    6057103|    4|
|   10805992|    1|
|   50621360|    2|
|   35465009|    1|
|    2784934|   34|
|   48396961|    1|
|    2456076|    1|
|   22163038|    1|
|   11831977|    1|
|    1513544|    1|
+-----------+-----+
only showing top 15 rows



In [19]:
customers = customers.withColumnRenamed("count", "customer_count")
customers.show(15)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   32605012|             2|
|    7458575|             1|
|   33739415|            22|
|   52884959|             3|
|   18141648|             1|
|    6057103|             4|
|   10805992|             1|
|   50621360|             2|
|   35465009|             1|
|    2784934|            34|
|   48396961|             1|
|    2456076|             1|
|   22163038|             1|
|   11831977|             1|
|    1513544|             1|
+-----------+--------------+
only showing top 15 rows



In [20]:
vine_table = shoes_data_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show(15)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1000E0AHJ5E1V|          4|            0|          0|   N|
|R100A5CHCA9GMG|          1|            2|          2|   N|
|R100A968SZR4GH|          5|            0|          0|   N|
|R100HVIG9VEABL|          5|            1|          1|   N|
|R100ZMBT0DGFFH|          5|            8|         10|   N|
|R101BSLZ8K12T6|          5|            0|          1|   N|
|R101I5DOJAUSZ7|          4|            0|          0|   N|
|R101VVQ9LXBADV|          5|            1|          1|   N|
|R101VY19JHVABI|          4|            0|          0|   N|
|R1026R3QQEM02Q|          5|            0|          0|   N|
|R102IW3CHZL3F4|          5|            0|          0|   N|
|R102JNG6P6AHY7|          5|            0|          0|   N|
|R102X52FTDKNGJ|          4|            0|          0|   N|
|R10330PL0P9BSY|          4|            

In [26]:
mode = "append"
jdbc_url="jdbc:postgresql://<connection string>:5432/<database-name>"
config = {"user":"postgres", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [27]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)